In [1]:
import neo4j
import jsonlines
import csv
import pandas as pd
from py2neo import Graph, Node, Relationship
import re
import logging
from tqdm import tqdm
import networkx as nx
import os
os.chdir("A:\Desktop\COMP7600\dataset_label_fine-tune")
import glob
import pandas as pd
import matplotlib.pyplot as plt
import faiss
from transformers import ( 
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
import torch
import numpy as np



b:\Conda\envs\basic3\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
uri = "bolt://192.168.0.117:7687"
username = "neo4j"
password = "Tzmt541881"

graph = Graph(uri, auth=(username, password))

In [1]:
def load_data(file_path):
    data = []
    for file in glob.glob(os.path.join(file_path, "*.jsonl")):
        with open(file, 'r', encoding='utf-8') as f:
            for item in jsonlines.Reader(f):
                data.append(item)
    return data

def filter_data(data):
    entity_dict = ['frequency', 'examination', 'treatment', 'causes', 'materials', 'population', 'symptom', 'usage', 'clinical manifestations', \
        'medication information', 'dosage', 'instrument', 'oral part', 'description', 'prevention', 'disease', 'medicine']
    title_list = ['relation_type', 'Entity1_type', 'Entity2_type', 'Entity1_name', 'Entity2_name']
    relation_dict = {
        'medicine_contraindicates_population': ['medicine', 'population'], 
        'treatment_has_description': ['treatment', 'description'], 
        'is': [], 
        'medicine_contraindication_disease': ['medicine', 'disease'], 
        'disease_caused_disease': ['disease', 'disease'], 
        'symptom_has_description': ['symptom', 'description'], 
        'medicine_side-effect_symptom': ['medicine','symptom'], 
        'disease_usually happens_population': ['disease', 'population'], 
        'disease_has_clinical manifestations': [], 
        'clinical manifestations_happens at_oral part': ['clinical manifestations', 'oral part'], 
        'causes_disease': ['causes', 'disease'], 
        'symptom_oral part': ['symptom', 'oral part'], 
        'clinical manifestations_use_treatment': ['clinical manifestations', 'treatment'], 
        'examination_has_description': ['examination', 'description'], 
        'symptom_cause_symptom': ['symptom', 'symptom'], 
        'medicine_medication-information': ['medicine','medication-information'], 
        'disease_has_description': ['disease', 'description'], 
        'oral part_discription': ['oral part', 'discription'],  
        'examination_at_oral part': ['examination', 'oral part'], 
        'medicine_side-effect_disease': ['medicine', 'disease'], 
        'disease_symptom': ['disease', 'symptom'], 
        'disease_oral part': ['disease', 'oral part'], 
        'disease_examination': ['disease', 'examination'], 
        'prevent': [], 
        'disease_treatment': ['disease', 'treatment'], 
        'medicine_has_description': ['medicine', 'description'], 
        'disease_lack response_examination': ['disease', 'examination'], 
        'treatment_frequency': ['treatment', 'frequency'], 
        'medicine_reduce_symptom': ['medicine','symptom'], 
        'materials_use_as_medicine': ['materials', 'medicine'], 
        'instrument_discription': ['instrument', 'discription'], 
        'parent-child': [], 
        'medicine_treats_disease': ['medicine', 'disease'], 
        'clinical manifestations_from_examination': ['clinical manifestations', 'examination'], 
        'materials_has_description': ['materials', 'description']
        }
    check_list = ['medicine_medication-information', 'population_use_as_alternative_medicine', 'medication-information_frequency']
    title_list = ['relation_type', 'Entity1_type', 'Entity1_name', 'Entity2_type', 'Entity2_name']
    basic_dict = []
    for item in data:
        triples = item['response']
        for triple in triples: 
            # print(item['id'],triple)
            if any(item not in title_list for item in triple.keys()) or any(item not in triple.keys() for item in title_list)\
                or not all(item in triple.keys() for item in title_list) or any(item == None for item in triple.values()):
                continue

            triple['relation_type'] = triple['relation_type'].replace('medication-information_frequency', 'medication information_frequency')\
                            .replace('Causes_disease', 'causes_disease').replace('symptom_oralpart', 'symptom_oral part')\
                                .replace('disease_oral-part', 'disease_oral part').replace('materials_use_as_medicine', 'materials_use as_medicine').lower()
                
            triple['Entity1_type'] = str(triple['Entity1_type']).replace('Causes', 'causes').lower()
            triple['Entity2_type'] = str(triple['Entity2_type']).replace('Causes', 'causes').lower()
            triple['Entity1_name'] = str(triple['Entity1_name']).lower()
            triple['Entity2_name'] = str(triple['Entity2_name']).lower()
            
            # if not all(item in triple.keys() for item in title_list):
                # continue

            # if any(item == None for item in triple.values()):
                # continue

            if triple['relation_type'] not in check_list and triple['relation_type'] in relation_dict.keys()\
                and triple['Entity1_name'] not in entity_dict and triple['Entity2_name'] not in entity_dict\
                    and triple['Entity1_type'] in entity_dict and triple['Entity2_type'] in entity_dict:
                if relation_dict[triple['relation_type']] == []:
                    basic_dict.append({
                        'source': item['id'],
                        'data': triple
                    })
                else:
                    if (triple['Entity2_type'] == relation_dict[triple['relation_type']][0] \
                        and triple['Entity1_type'] == relation_dict[triple['relation_type']][1])\
                         or (triple['Entity1_type'] == relation_dict[triple['relation_type']][0] \
                            and triple['Entity2_type'] == relation_dict[triple['relation_type']][1]):
                        basic_dict.append({
                            'source': item['id'],
                            'data': triple
                        })
    return basic_dict

def clean_string(s):
    return re.sub(r'\s+', ' ', s.strip())

def get_or_create_entity(entity_id, entity_type, entity_name, entity_source):
    if entity_type not in entities_dict:
        entities_dict[entity_type] = {}
    
    if entity_name not in entities_dict[entity_type]:
        # 创建实体
        node = Node(entity_type, id=entity_id, name=entity_name, source=entity_source, )
        graph.create(node)
        entities_dict[entity_type][entity_name] = node
        logging.info(f'Entity node created: {entity_type}_{entity_name}.')
        return node
    else:
        # 获取已存在的实体节点
        node = entities_dict[entity_type][entity_name]
        logging.info(f'Entity already exists: {entity_type}_{entity_name}!')

        # 追加新的 source 信息
        if 'source' in node:
            existing_source = node['source']
            if isinstance(existing_source, list):
                existing_source.append(entity_source)
            else:
                node['source'] = f"{existing_source}; {entity_source}"
        else:
            node['source'] = entity_source

        # 保存更新后的节点
        graph.push(node)
        logging.info(f'Entity source updated for {entity_type}_{entity_name}.')
        
        return node

def upload_neo4j(df):
    for index, row in tqdm(df.iterrows(), total=len(df)):
        x_node = get_or_create_entity(row['x_id'], row['subject_type'], row['subject'].replace('’', "'"), row['source'])
        y_node = get_or_create_entity(row['y_id'], row['object_type'], row['object'].replace('’', "'"), row['source'])

        relationship = Relationship(x_node, row['relation'], y_node)
        graph.create(relationship)
        logging.info(f"Relationship created: {row['relation']}.")

def id_check(id_table, name, i):
    if name not in id_table.keys():
        id_table[name] = i
        return id_table, i + 1
    else:
        return id_table, i

def convert_to_csv(kg_path, basic_dict):
    with open(kg_path, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['x_id', 'subject', 'subject_type', 'relation', 'y_id', 'object', 'object_type', 'source']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        id_table = {}
        writer.writeheader()
        i = 1
        for triplet in basic_dict:
            source = triplet['source']
            triplet = triplet['data']
            
            id_table, i = id_check(id_table, triplet['Entity1_name'], i)
            id_table, i = id_check(id_table, triplet['Entity2_name'], i)

            # print(id_table)
            writer.writerow({
                'x_id': id_table[triplet['Entity1_name']],
                'subject': triplet['Entity1_name'],
                'subject_type': triplet['Entity1_type'],
                'relation': triplet['relation_type'],
                'y_id': id_table[triplet['Entity2_name']],
                'object': triplet['Entity2_name'],
                'object_type': triplet['Entity2_type'],
                'source': source
            })
        return id_table
        
def load_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
    model = AutoModelForCausalLM.from_pretrained(
        './models/Llama3-Med42-8B',
        device_map={"":0},
        trust_remote_code=True,
        quantization_config=bnb_config
    )

    tokenizer = AutoTokenizer.from_pretrained('./models/Llama3-Med42-8B')
    # tokenizer.pad_token = tokenizer.eos_token
    return model, tokenizer
    

In [5]:
file_path = './extraction_R1'
kg_path = './kg1.csv'
id_table = convert_to_csv(kg_path, filter_data(load_data(file_path)))


In [3]:

query = """
MATCH (n)
RETURN COUNT(DISTINCT n) AS distinct_entities_count"""
result = graph.run(query)
print(result)
# with open('test.txt', 'w') as f:
#     f.write(str(result))

 distinct_entities_count 
-------------------------
                    9464 



: 

In [ ]:
# 查询数据
query = """
MATCH (n)-[r]->(m)
RETURN n, r, m
LIMIT 50
"""
data = graph.run(query).data()

# 创建NetworkX图
G = nx.DiGraph()

# 添加节点和边
for record in data:
    n = record['n']
    m = record['m']
    r = record['r']
    G.add_node(n['name'], label=n['name'])
    G.add_node(m['name'], label=m['name'])
    G.add_edge(n['name'], m['name'], label=r['name'])

# 绘制图形
pos = nx.spring_layout(G)  # 使用Spring布局
labels = nx.get_edge_attributes(G, 'label')

plt.figure(figsize=(12, 8))
nx.draw(G, pos, with_labels=True, node_size=3000, node_color='lightblue', font_size=10, font_weight='bold', arrows=True)
nx.draw_networkx_edge_labels(G, pos, edge_labels=labels, font_color='red')

plt.title("Neo4j Graph Visualization")
plt.show()

In [7]:
# 清除数据库中的所有数据
graph.run("MATCH (n) DETACH DELETE n")

print("所有数据已清除")

所有数据已清除


In [10]:
csv_file = './kg1.csv'
df = pd.read_csv(csv_file)
df = df.applymap(lambda x: clean_string(x) if isinstance(x, str) else x)

entities_dict = {}
logging.basicConfig(filename='neo4j_import.log', level=logging.INFO, format='%(asctime)s %(message)s')

upload_neo4j(df)

100%|██████████| 14910/14910 [21:46<00:00, 11.41it/s]


: 

In [11]:
model, tokenizer =load_model_and_tokenizer()

Loading checkpoint shards: 100%|██████████| 4/4 [00:31<00:00,  7.97s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
loaded_embeddings = np.load('./combined_entity_embeddings.npy')


In [9]:
print(len(entities_list))

8115


In [14]:
ids_list = []
entities_list = []
for item, id in id_table.items():
    ids_list.append(id)
    entities_list.append(item)

entities_ids = np.array(ids_list, dtype=np.int64)

In [15]:
print(ids_list[573:575])
print(entities_list[573:575])

[574, 575]
['bony defect', 'deep pocketing']


In [14]:
id_table_list = [{'name': entities_list[i], 'id': ids_list[i]} for i in range(len(ids_list))]

In [15]:
id_table_list = [{'name': entities_list[i], 'id': ids_list[i]} for i in range(len(ids_list))]
df = pd.DataFrame(id_table_list)
df.to_csv('table.csv', index=False)

In [7]:
def query_to_embedding(query):
    inputs = tokenizer(query, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    outputs = model(**inputs)
    cls_embedding = outputs.hidden_states[-1][:, 0, :].numpy()
    return cls_embedding

def search_similar_entities(query, index, entities, top_k=5):
    query_embedding = query_to_embedding(query)
    D, I = index.search(query_embedding, top_k)
    similar_entities = [entities[i] for i in I[0]]
    return similar_entities

dimension = loaded_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index_with_ids = faiss.IndexIDMap(index)
index_with_ids.add_with_ids(loaded_embeddings, entities_ids)


In [8]:
index.ntotal

6865

In [73]:

question_embedding = np.load('./quesitons (1).npy')
temp_1 = np.load('./combined_entity_embeddings.npy')
temp_2 = temp_1[5217]
print(question_embedding)

[[ 2.549   0.2358 -0.902  ... -1.55    0.322   2.422 ]
 [ 2.549   0.234  -0.9014 ... -1.552   0.3215  2.422 ]]


In [74]:
l2_distance = np.linalg.norm(question_embedding[0] - temp_1)
print(l2_distance)

0.0


In [66]:
query_vector = []
for question in question_embedding:
    query_vector.append(np.array([question], dtype=np.float32))
print(query_vector)

[array([[ 2.5488281 ,  0.23583984, -0.90185547, ..., -1.5498047 ,
         0.32202148,  2.421875  ]], dtype=float32), array([[ 2.5488281 ,  0.23400879, -0.9013672 , ..., -1.5517578 ,
         0.3215332 ,  2.421875  ]], dtype=float32)]


In [67]:
distances, result_ids = index_with_ids.search(query_vector[0], k=5)
print("Distances:", distances)
print("Resulting IDs:", result_ids)

Distances: [[0.02630468 0.02630468 0.02630468 0.02630468 0.02648782]]
Resulting IDs: [[1528 1664 3859 6278   13]]


In [68]:
for i in result_ids[0]:
    print(entities_list[int(i)])

di-calcium silicate
enamel lost rapidly due to defective edj brown; opalescent dentine colour; prone to fracture/wear short roots, bulbous crowns; pulps obliterated
neurosensory impairment
vital pulp therapy
external cervical resorption


In [5]:
from sentence_transformers import SentenceTransformer
embedder_model = SentenceTransformer("A:\Desktop\COMP7600\dataset_label_fine-tune/models/pubmedbert-base-embeddings")

def get_entity_embeddings(entities):
    embeddings = []
    for entity in tqdm(entities):
        output_embedding = embedder_model.encode(entity)
        embeddings.append(output_embedding)
    return np.vstack(embeddings)

question_list = ['A biopsy specimen of the lower lip salivary glands showed replacement of parenchymal tissue by lymphocytes. The patient also had xerostomia and eratoconjunctivitis sicca. These findings are indicative of which disease?']
# question_list = ['xerostomia', 'keratoconjunctivitis sicca']

question_embedding = get_entity_embeddings(question_list)
# print(question_embedding)

query_vector = []
for question in question_embedding:
    query_vector.append(np.array([question], dtype=np.float32))

# l2_distance = np.linalg.norm(question_embedding[0] - question_embedding[1])
# print(l2_distance)

distances, result_ids = index_with_ids.search(query_vector[0], k=5)
print("Distances:", distances)
print("Resulting IDs:", result_ids)

for i in result_ids[0]:
    print(entities_list[int(i-1)])

  0%|          | 0/1 [00:00<?, ?it/s]b:\Conda\envs\basic3\lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
100%|██████████| 1/1 [00:00<00:00,  1.55it/s]


NameError: name 'index_with_ids' is not defined

In [7]:
np.save('question.npy', question_embedding)

: 

In [ ]:
from sentence_transformers import SentenceTransformer
embedder_model = SentenceTransformer("./pubmedbert-base-embeddings")

def get_entity_embeddings(entities):
    embeddings = []
    for entity in tqdm(entities):
        output_embedding = embedder_model.encode(entity)
        embeddings.append(output_embedding)
    return np.vstack(embeddings)

question_list = ['xerostomia', 'keratoconjunctivitis sicca']

question_embedding = get_entity_embeddings(question_list)

In [ ]:
from scipy.spatial.distance import cosine


cosine_sim = 1 - cosine(test[0], test[1])

def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    
    cosine_sim = dot_product / (norm_vec1 * norm_vec2)
    
    return cosine_sim

l2_distance = cosine_similarity(test[0], test[1])
print(cosine_sim)